In [1]:
import tensorflow as tf
import numpy as np
import json

# === 資料預處理 ===
# 載入 Fashion-MNIST 資料集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 正規化像素值到 0~1 範圍
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 可選：分割驗證集（20% 訓練集作為驗證）
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# === 模型建構 ===
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # 輸入層展平
    tf.keras.layers.Dense(256, activation='relu'),  # 隱藏層1
    tf.keras.layers.Dense(128, activation='relu'),  # 隱藏層2
    tf.keras.layers.Dense(10, activation='softmax') # 輸出層
])

# 編譯模型
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# === 模型訓練 ===
history = model.fit(
    x_train, y_train,
    epochs=15,
    batch_size=256,
    validation_data=(x_val, y_val)
)

# === 模型評估 ===
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'\n測試集準確率: {test_acc:.4f}')

# === 儲存模型 ===
# 儲存為 .h5 格式
model.save('fashion_mnist.h5')

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7215 - loss: 0.8102 - val_accuracy: 0.8473 - val_loss: 0.4264
Epoch 2/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8568 - loss: 0.4085 - val_accuracy: 0.8636 - val_loss: 0.3818
Epoch 3/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8765 - loss: 0.3434 - val_accuracy: 0.8711 - val_loss: 0.3535
Epoch 4/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8853 - loss: 0.3175 - val_accuracy: 0.8706 - val_loss: 0.3514
Epoch 5/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8879 - loss: 0.3048 - val_accuracy: 0.8805 - val_loss: 0.3304
Epoch 6/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.8961 - loss: 0.2847 - val_accuracy: 0.8832 - val_loss: 0.3210
Epoch 7/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9030 - loss: 0.2642 - val_accuracy: 0.8724 - val_loss: 0.3535
Epoch 8/15
188/188 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9009 - loss: 0.2637 - val_accu


測試集準確率: 0.8881


In [2]:
YOUR_MODEL_NAME = 'fashion_mnist' # Default extension is h5
TF_MODEL_PATH = f'{YOUR_MODEL_NAME}.h5'
MODEL_WEIGHTS_PATH = f'{YOUR_MODEL_NAME}.npz'
MODEL_ARCH_PATH = f'{YOUR_MODEL_NAME}.json'

In [3]:
model = tf.keras.models.load_model(TF_MODEL_PATH)

# Save weights to .npz (NumPy format)
weights = model.get_weights()
np.savez('model_weights.npz', *weights)

# Save architecture to JSON
with open('model_architecture.json', 'w') as json_file:
    json_file.write(model.to_json())

In [4]:
import tensorflow as tf
import numpy as np

# === Step 1: Load Keras .h5 model ===
model = tf.keras.models.load_model(TF_MODEL_PATH)

# === Step 2: Print and collect weights ===
params = {}
print("🔍 Extracting weights from model...\n")
for layer in model.layers:
    weights = layer.get_weights()
    if weights:
        print(f"Layer: {layer.name}")
        for i, w in enumerate(weights):
            param_name = f"{layer.name}_{i}"
            print(f"  {param_name}: shape={w.shape}")
            params[param_name] = w
        print()

# === Step 3: Save to .npz ===
np.savez(MODEL_WEIGHTS_PATH, **params)
print(f"✅ Saved all weights to {MODEL_WEIGHTS_PATH}")

# === Step 4: Reload and verify ===
print("\n🔁 Verifying loaded .npz weights...\n")
loaded = np.load(MODEL_WEIGHTS_PATH)

for key in loaded.files:
    print(f"{key}: shape={loaded[key].shape}")

# === Step 6: Extract architecture to JSON ===
arch = []
for layer in model.layers:
    config = layer.get_config()
    info = {
        "name": layer.name,
        "type": layer.__class__.__name__,
        "config": config,
        "weights": [f"{layer.name}_{i}" for i in range(len(layer.get_weights()))]
    }
    arch.append(info)

with open(MODEL_ARCH_PATH, "w") as f:
    json.dump(arch, f, indent=2)

print("✅ Architecture saved to model_architecture.json")

🔍 Extracting weights from model...

Layer: dense
  dense_0: shape=(784, 256)
  dense_1: shape=(256,)

Layer: dense_1
  dense_1_0: shape=(256, 128)
  dense_1_1: shape=(128,)

Layer: dense_2
  dense_2_0: shape=(128, 10)
  dense_2_1: shape=(10,)

✅ Saved all weights to fashion_mnist.npz

🔁 Verifying loaded .npz weights...

dense_0: shape=(784, 256)
dense_1: shape=(256,)
dense_1_0: shape=(256, 128)
dense_1_1: shape=(128,)
dense_2_0: shape=(128, 10)
dense_2_1: shape=(10,)
✅ Architecture saved to model_architecture.json


## NN forward Path using Numpy only

In [5]:
import numpy as np
import json

# === Load weights and architecture ===
weights = np.load(MODEL_WEIGHTS_PATH)
with open(MODEL_ARCH_PATH) as f:
    architecture = json.load(f)


# === Activation functions ===
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e / np.sum(e, axis=-1, keepdims=True)

# === Flatten ===
def flatten(x):
    return x.reshape(x.shape[0], -1)

# === Dense layer ===
def dense(x, W, b):
    return x @ W + b

# === Forward pass ===
def forward(x):
    for layer in architecture:
        lname = layer['name']
        ltype = layer['type']
        cfg = layer['config']
        wnames = layer['weights']


        if ltype == "Flatten":
            x = flatten(x)

        elif ltype == "Dense":
            W = weights[wnames[0]]
            b = weights[wnames[1]]
            x = dense(x, W, b)
            if cfg.get("activation") == "relu":
                x = relu(x)
            elif cfg.get("activation") == "softmax":
                x = softmax(x)

    return x

# === Example usage ===
# Load a dummy image (batch size 1)
# Make sure it's shape: (1, 28, 28, 1)
dummy_input = np.random.rand(1, 28*28).astype(np.float32)
output = forward(dummy_input)

print("🧠 Output probabilities:", output)
print("✅ Predicted class:", np.argmax(output, axis=-1))

🧠 Output probabilities: [[8.42138519e-08 8.97192390e-12 1.12395874e-07 2.86170776e-09
  4.64359051e-10 2.10895309e-16 6.22713287e-07 2.26218042e-15
  9.99999166e-01 3.48914756e-12]]
✅ Predicted class: [8]
